In [1]:
import numpy as np
import sys
import re
import os
import pandas as pd
import warnings
from pathlib import Path
from typing import List, Dict, Tuple, Any, Union, Callable
from tqdm import tqdm
import csv
import json
from dataclasses import dataclass
from datetime import datetime
from configobj import ConfigObj, Section
import shutil
from tempfile import NamedTemporaryFile

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

In [2]:
WDIR = Path().cwd().parent

PLANET_DB_PATH = str(WDIR / "data/planet_database_composite.csv")
OPACITY_PATH = str(WDIR / "data/Input/xsec/xsec_sampled_R15000_0.3-15")
CIA_PATH = str(WDIR / "data/Input/cia/hitran")
KTABLE_PATH = str(WDIR / "data/Input/ktables/R100")
MOLECULE_PATH = str(WDIR / "data/molecule_db.json")

SPECTRA_BE_PATH = str(WDIR / "data/SpectraBE")
SPECTRA_LW_PATH = str(WDIR / "data/taurex_lightcurves_LW")

In [3]:

def make_par_file():
    pass

def run_retrieval():
    pass

In [ ]:
path_list = [
    str(WDIR / "data/taurex_lightcurves_LW" / "HAT-P-1-b_HST_STIS_G430L_52X2_Nikolov+2014.txt"),
    str(WDIR / "data/taurex_lightcurves_LW" / "HAT-P-1-b_HST_STIS_G430L_52X2_Sing+2016.txt"),
]

target = get_target_data("HAT-P-1 b")

write_par_file(path_list, target=target)

In [3]:
import numpy as np
import sys
import re
import os
import pandas as pd
import warnings
from pathlib import Path
from typing import List, Dict, Tuple, Any, Union, Callable
from tqdm import tqdm
import csv
import json
from dataclasses import dataclass

import matplotlib.pyplot as plt
import seaborn as sns
from corner import corner

plt.rcParams['figure.figsize'] = [7, 7]
plt.rcParams['figure.dpi'] = 300


import taurex.log
# taurex.log.disableLogging()
from taurex.cache import OpacityCache, CIACache
from taurex.data.profiles.temperature import Isothermal, Guillot2010, NPoint
from taurex.data import Planet
from taurex.data.stellar import PhoenixStar
from taurex.data.stellar import BlackbodyStar
from taurex.data.profiles.chemistry import TaurexChemistry
from taurex.data.profiles.chemistry import ConstantGas
from taurex.data.profiles.chemistry import TwoLayerGas
from taurex.model import TransmissionModel, EmissionModel
from taurex.contributions import AbsorptionContribution, CIAContribution, RayleighContribution, HydrogenIon
from taurex.data.spectrum.observed import ObservedSpectrum
from astropy.io.fits.verify import VerifyWarning
from taurex.binning.fluxbinner import FluxBinner
from taurex.util.util import wnwidth_to_wlwidth
from taurex.optimizer.nestle import NestleOptimizer


In [4]:
def show_plugins():
    from taurex.parameter.classfactory import ClassFactory
    from taurex.log import setLogLevel
    import logging
    setLogLevel(logging.ERROR)

    successful_plugins, failed_plugins = ClassFactory().discover_plugins()

    print('\nSuccessfully loaded plugins')
    print('---------------------------')
    for k, v in successful_plugins.items():
        print(k)

    print('\n\nFailed plugins')
    print('---------------------------')
    for k, v in failed_plugins.items():
        print(k)
        print(f'Reason: {v}')

    print('\n')

In [11]:
from taurex.parameter import ParameterParser
from taurex.output.hdf5 import HDF5Output
from taurex_offset import OffsetSpectra

print(OffsetSpectra.__bases__)

import inspect
print(inspect.getfile(OffsetSpectra))

path = str(WDIR / "data/retrievals/DEFAULT/default.par")

print(show_plugins())


pp = ParameterParser()
pp.read(path)

pp.setup_globals()

observation = pp.generate_observation()

binning = pp.generate_binning()

instrument = pp.generate_instrument(binner=binning)

tm = pp.generate_model()
tm.build()

print(tm.fittingParameters.keys())

opt = pp.generate_optimizer()

opt.set_model(tm)


pp.setup_optimizer(opt)

(<class 'taurex.data.spectrum.array.ArraySpectrum'>, <class 'taurex.data.fittable.Fittable'>)
C:\Users\lukas\anaconda3\envs\compla\lib\site-packages\taurex_offset\offset.py

Successfully loaded plugins
---------------------------
taurex_offset


Failed plugins
---------------------------


None
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\lukas\anaconda3\envs\compla\lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\lukas\AppData\Local\Temp\ipykernel_6976\2256283754.py", line 20, in <module>
    observation = pp.generate_observation()
  File "C:\Users\lukas\anaconda3\envs\compla\lib\site-packages\taurex\parameter\parameterparser.py", line 247, in generate_observation
    return create_observation(observation_config)
  File "C:\Users\lukas\anaconda3\envs\compla\lib\site-packages\taurex\parameter\factory.py", line 369, in create_observation
    obj = klass(**config)
  File "C:\Users\lukas\anaconda3\envs\compla\lib\site-packages\taurex_offset\offset.py", line 52, in __init__
    self.generate_offset_fitting_params()
  File "C:\Users\lukas\anaconda3\envs\compla\lib\site-packages\taurex_offset\offset.py", line 71, in generate_offset_fitting_params
    self.add_fittable_param(para

In [54]:
print(opt.fitting_parameters)

[]


In [40]:
# pp.setup_optimizer(opt)

# fitting_parameters = pp.generate_fitting_parameters()

 # NestleOptimizer(num_live_points=50)


# print(fitting_parameters)
print(tm.fittingParameters)

{'planet_mass': ('planet_mass', '$M_p$', <bound method fitparam.<locals>.wrapper of <taurex.data.planet.Planet object at 0x0000024CAD7D63A0>>, <bound method BasePlanet.mass of <taurex.data.planet.Planet object at 0x0000024CAD7D63A0>>, 'linear', False, [0.5, 1.5]), 'planet_radius': ('planet_radius', '$R_p$', <bound method fitparam.<locals>.wrapper of <taurex.data.planet.Planet object at 0x0000024CAD7D63A0>>, <bound method BasePlanet.radius of <taurex.data.planet.Planet object at 0x0000024CAD7D63A0>>, 'linear', True, [0.9, 1.1]), 'planet_distance': ('planet_distance', '$D_{planet}$', <bound method fitparam.<locals>.wrapper of <taurex.data.planet.Planet object at 0x0000024CAD7D63A0>>, <bound method BasePlanet.distance of <taurex.data.planet.Planet object at 0x0000024CAD7D63A0>>, 'linear', False, [1, 2]), 'planet_sma': ('planet_sma', '$D_{planet}$', <bound method fitparam.<locals>.wrapper of <taurex.data.planet.Planet object at 0x0000024CAD7D63A0>>, <bound method BasePlanet.semiMajorAxis o

In [42]:
print(tm.fitting_parameters)

opt.set_model(tm)



# opt.update_model(tm.fittingParameters)
pp.setup_optimizer(opt)


taurex.ParamParser - INFO - Setting up optimizer


<bound method Fittable.fitting_parameters of <taurex.model.transmission.TransmissionModel object at 0x0000024CADB7CF70>>
Offset_1
{'fit': False, 'bounds': None, 'mode': None, 'factor': None, 'prior': <taurex.core.priors.Uniform object at 0x0000024CADBAC3A0>}

Slope_1
{'fit': False, 'bounds': None, 'mode': None, 'factor': None, 'prior': <taurex.core.priors.Uniform object at 0x0000024CADBAC250>}

Offset_2
{'fit': True, 'bounds': None, 'mode': None, 'factor': None, 'prior': <taurex.core.priors.Uniform object at 0x0000024CADBAC910>}

Slope_2
{'fit': True, 'bounds': None, 'mode': None, 'factor': None, 'prior': <taurex.core.priors.Uniform object at 0x0000024CADBACA90>}

H2O
{'fit': True, 'bounds': None, 'mode': None, 'factor': None, 'prior': <taurex.core.priors.LogUniform object at 0x0000024CADBACDC0>}

CH4
{'fit': True, 'bounds': None, 'mode': None, 'factor': None, 'prior': <taurex.core.priors.LogUniform object at 0x0000024CADBACE80>}

NH3
{'fit': True, 'bounds': None, 'mode': None, 'factor

Traceback (most recent call last):
  File "C:\Users\lukas\anaconda3\envs\compla\lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\lukas\AppData\Local\Temp\ipykernel_10840\164745783.py", line 8, in <module>
    pp.setup_optimizer(opt)
  File "C:\Users\lukas\anaconda3\envs\compla\lib\site-packages\taurex\parameter\parameterparser.py", line 196, in setup_optimizer
    optimizer.disable_fit(key)
  File "C:\Users\lukas\anaconda3\envs\compla\lib\site-packages\taurex\optimizer\optimizer.py", line 320, in disable_fit
    self._model.fittingParameters[parameter]
KeyError: 'Offset_1'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\lukas\anaconda3\envs\compla\lib\site-packages\IPython\core\interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\lukas\anaconda3\envs\co

In [ ]:
import sys
from taurex.log import setLogLevel
from taurex.cache import OpacityCache
from taurex.parameter import ParameterInput
from taurex.retrieval import Retrieval
from taurex.output import OutputFile


from taurex.parameter import P
# Set log level
setLogLevel('info')

# Load input .par file
input_file = 'quickstart.par'
parameter_input = ParameterInput.from_file(input_file)

# Initialize opacity cache
opacity_cache = OpacityCache()

# Create a retrieval object
retrieval = Retrieval.from_parameter_input(parameter_input, opacity_cache)

# Run the retrieval
retrieval()

# Set output .hdf5 file
output_file = 'myfile_retrieval.hdf5'

# Save the output
with OutputFile.create(output_file, retrieval) as of:
    of.write()

# Plot the results (optional)
if parameter_input.retrieval.plot:
    retrieval.plot()

print(f"Retrieval complete. Results saved to: {output_file}")